In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image




# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
sys.path.append("/home/nandagopal/PycharmProjects/models/research")
from object_detection.utils import ops as utils_ops



if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

In [2]:

# This is needed to display the images.
%matplotlib inline

In [3]:
from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

In [4]:
MODEL_NAME = 'inference_graph'
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = 'training/object-detection.pbtxt'

In [5]:

# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#   file_name = os.path.basename(file.name)
#   if 'frozen_inference_graph.pb' in file_name:
#     tar_file.extract(file, os.getcwd())

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [7]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [8]:
def run_inference_for_single_image(image, graph):
    if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
    image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

    # Run inference
    output_dict = sess.run(tensor_dict,
                            feed_dict={image_tensor: np.expand_dims(image, 0)})

    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict[
        'detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [15]:
import cv2
cap = cv2.VideoCapture(0)
print("Web cam on...", cap)
try:
    with detection_graph.as_default():
        with tf.Session() as sess:
                # Get handles to input and output tensors
                ops = tf.get_default_graph().get_operations()
                all_tensor_names = {output.name for op in ops for output in op.outputs}
                tensor_dict = {}
                for key in [
                  'num_detections', 'detection_boxes', 'detection_scores',
                  'detection_classes', 'detection_masks'
                ]:
                    tensor_name = key + ':0'
                    if tensor_name in all_tensor_names:
                        tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                      tensor_name)
                print("While Loop start")
                m = 0
                while m<2:
                    m += 1
                    img_path = "/home/nandagopal/PycharmProjects/pic2code/obj_detection/images/test/test2.png"
                    image_np = cv2.imread(img_path)
#                     print("Started Reading from web cam...", image_np)
                    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                    image_np_expanded = np.expand_dims(image_np, axis=0)
                    # Actual detection.
                    output_dict = run_inference_for_single_image(image_np, detection_graph)
                    
                    print( "Visualization of the results of a detection", output_dict)
                    vis_util.visualize_boxes_and_labels_on_image_array(
                        image_np,
                        output_dict['detection_boxes'],
                        output_dict['detection_classes'],
                        output_dict['detection_scores'],
                        category_index,
                        instance_masks=output_dict.get('detection_masks'),
                        use_normalized_coordinates=True,
                        line_thickness=8)
                    print("Imshow")
#                     cv2.imshow('object_detection', cv2.resize(image_np, (800, 600)))
                    cv2.imwrite('detected_1.png', image_np)
                    print("Image show here....")
                    if cv2.waitKey(25) & 0xFF == ord('q'):
                        cap.release()
                        cv2.destroyAllWindows()
                        break
except Exception as e:
    print("Exception")
    print(e)
    cap.release()

Web cam on... <VideoCapture 0x7f13b05eeb10>
While Loop start
Visualization of the results of a detection {'num_detections': 300, 'detection_boxes': array([[0.51056427, 0.36161628, 0.55705184, 0.9024821 ],
       [0.41544545, 0.04888057, 0.4861175 , 0.2995095 ],
       [0.12555742, 0.31895143, 0.1921728 , 0.91001743],
       ...,
       [0.58064693, 0.02669933, 0.599139  , 0.08309989],
       [0.5412059 , 0.3766921 , 0.61859417, 0.9859198 ],
       [0.        , 0.        , 0.58766514, 0.84108806]], dtype=float32), 'detection_scores': array([9.74137425e-01, 7.92619765e-01, 5.56659937e-01, 5.05285501e-01,
       3.32734585e-01, 3.17536354e-01, 2.32552096e-01, 2.08011940e-01,
       1.83821157e-01, 1.65181756e-01, 1.63753793e-01, 1.56226575e-01,
       1.52886987e-01, 1.39578551e-01, 1.38751566e-01, 1.15613319e-01,
       1.08885251e-01, 1.04613155e-01, 7.55552649e-02, 4.74964716e-02,
       4.55849022e-02, 4.31634262e-02, 3.52590978e-02, 2.68780105e-02,
       2.41902322e-02, 2.19637249e-